In [119]:
SAMPLE = True
SAMPLE_SUFFIX = '_dummy' # for input and output files

#  Lexical Resource (LR) for output

#### Imports

In [91]:
# interface
from tqdm import tqdm
from dsutils.de.files import dbg

## Files and filesystem
import os
import json
import csv
import glob

## Data management
import numpy as np
import pandas as pd

## DS tools
import re
from types import NoneType
from dsutils.de.files import describe_csv, get_csv_head, xls_to_csv, get_data_path, get_datafile_path
from dsutils.nlp.language import get_lang_code

#### Paths

In [92]:
## input
db_paths = dict(
    eppo_com = get_datafile_path('original/2022-09-02_COMMONnames_EPPO_OQ.csv'),
    eppo_sci = get_datafile_path('original/2022-09-02_SCIENTIFICnames_EPPO_OQ.csv'),
    efsa = get_datafile_path('efsa_keyword_match/FichierMotsClesMagaliLarenaudie.csv'),
    ncbi = get_datafile_path('original/taxa+id_full.txt'), # or taxa+id_microbes+insects.txt
)

glossary_paths = dict(
    efsa_glossary = get_datafile_path('efsa_glossary.csv'),
    eppo_glossary = get_datafile_path('eppo_glossary.csv'),
)

## Output
data_path = get_data_path()
full_eppo_path = os.path.join(data_path, 'eppo_glossary.csv')
output_path = os.path.join(data_path, 'output')
output_gloss_path = os.path.join(data_path, 'output_glossary.csv')
LR_filepath = 'LR.csv'

#### Definitions

Defining functions for preprocessing of original data

In [93]:
def PHT_to_lower_taxon(PHT_code):
    taxon = re.sub(r'(?<!^)(?=[A-Z])', ' ', PHT_code[:-4]).lower()
    return taxon
# print('AcalymmaVittatum-PHT:',PHT_to_lower_taxon('AcalymmaVittatum-PHT')) # test
assert PHT_to_lower_taxon('AcalymmaVittatum-PHT') == 'acalymma vittatum'

def to_pattern(syn):
    r = syn.replace('+', ' ')
    r = r.replace('_', '\\w')
    if r.endswith('%'):
        r = r[:-1] + '\\w*'
    r = r.replace('% ', '\\w* ')
    r = r.replace('%', '\\S*\\s?')
    return r
assert to_pattern(r'chrysom_le%+ray_e+du+concombre') == 'chrysom\wle\w* ray\we du concombre'

def fill_values_cross_db(main_df,
                        lookup_df,
                        matching_columns = ('',''),
                        lookup_col = ''
                        ):
    if lookup_col not in main_df.columns:
        main_df[lookup_col] = ['']*len(main_df)
     
    for row_i in range(len(main_df)):
        main_df_key = main_df.at[row_i,matching_columns[0]]
        for lookup_row_i in range(len(lookup_df)):
            lookup_df_key = lookup_df.at[lookup_row_i,matching_columns[1]]
            if len(main_df_key) and len(lookup_df_key):
                if main_df_key == lookup_df_key or re.fullmatch(main_df_key, lookup_df_key):
                    cell_value = lookup_df.at[lookup_row_i,lookup_col]
                    main_df.at[row_i,lookup_col] = cell_value
    
    return main_df

#### Parameters

In [94]:
## parameters to read ncbi file
ncbi_sep = '\t'
ncbi_header = ['synonym','taxid','sci_name','tax_path','POS','rank','_', '__']

## which ncbi file to use (specifying extension)
ncbi_ext = '.csv'
db_paths['ncbi'] = db_paths['ncbi'][:-(len(ncbi_ext))]+ncbi_ext

## use a shorter sample file for testing 
if SAMPLE:
    for k in db_paths.keys():
        db_paths[k] = db_paths[k][:-4]+SAMPLE_SUFFIX+db_paths[k][-4:]
    LR_filepath = LR_filepath[:-4]+SAMPLE_SUFFIX+LR_filepath[-4:]

## name of the desired coluimns in the output LR
output_gloss_column_names = ['EPPO_CODE', 'EFSA_PHT', 'NCBI_TAXID', # 3 partial keys to relative dataset
                           'preferred_name', 'synonym', 're',# terms
                           'ds_language', 'match_language'] # languages

In [95]:
## column in each of the original datasets corresponding to the output LR column 
db_names_to_relevant_col_names = dict(
    eppo_com = ['CodeEOPP', None, None,
                'PreferredName', 'CommonName', None,
                ['Language', get_lang_code], None], # [obj (col name), fun] tuple when information is extracted via a function
    eppo_sci = ['CodeEOPP', None, None,
                'PreferredName', 'OtherScientificNames', None,
                None, None],
    efsa = [None, 'Category (pest name)', None,
                ['Category (pest name)', PHT_to_lower_taxon], None,  ['Keywords', to_pattern],
                None, 'Unnamed: 3'],
    ncbi = [None, None, 'taxid',
            'sci_name', 'synonym', None,
            None, None],
    )

#### Check Data

In [96]:
db_paths['eppo_com']

'/home/elubrini/GitHub/bio-corpus-translation/data/original/2022-09-02_COMMONnames_EPPO_OQ_sample.csv'

In [97]:
for db_name, path in db_paths.items():
    dbg(len(describe_csv(path).columns))
    if (len(describe_csv(path).columns))<=2:
        dbg(path)
        display(describe_csv(path))
        print('starting csv format standardisation:')
        normalised_df = pd.read_csv(path, names=ncbi_header, index_col=0, on_bad_lines='skip', sep=ncbi_sep, keep_default_na=False).reset_index(level=0)
        print('standardised:')
        display(normalised_df.head())
        new_path = path[:-4]+'.csv'
        
        normalised_df.to_csv(new_path, index=False,)
        db_paths[db_name] = new_path
    display(pd.read_csv(path, keep_default_na=False).head())

eppo_efsa_final_db_i: 5


,CommonName,Language,CodeEOPP,PreferredName,AuthorityPreferredName
0,A,,Acode,Asci,Asci
1,B,Bengalese,Bcode,Bsci,Bsci
2,Bb,Bengalese,Bcode,Bsci,Bsci
3,C,Congolese,Ccode,Csci,Csci


eppo_efsa_final_db_i: 5


,OtherScientificNames,Authority,CodeEOPP,PreferredName,AuthorityPreferredName
0,Z,authZ,Zcode,Zsci,Zsci


_13: 4


,Category (pest name),Keywords,Unnamed: 2,Unnamed: 3
0,Apht,A,,Albanian
1,Apht,X,,
2,Bpht,B,,
3,Dpht,D,,


prep_efsa_db_i: 3


,taxid,sci_name,synonym
0,Ataxid,Asci,A
1,Dtaxid,Dsci,D


# Create LR

### (1) Preprocess original datasets

Extract relevant information from each dataset and store it in table form (one `csv` per original dataset)

#### create dictionary of databases
Each database in the dictionary corresponds to one of the original databases, but containing only the relevant columns, renamed to match the output datatabase

In [98]:
db = dict() 

for db_name, columns in db_names_to_relevant_col_names.items(): # loop over db names and respective relevant columns
    db[db_name] = dict()    # new database (in dict of columns format) in db dictionary
    colum_renames_zip = zip(columns, output_gloss_column_names)  # match source-db column names to final-db column names

    ## iterate through columns in source dbs, preprocess them and use them to populate corresponding preprocessed database
    for orig_col, targ_col in colum_renames_zip: # loop over column renaming pairs
        if isinstance(orig_col, str): # if only column name specified (as opposed to col name + function), take column as is
            def fun(x):
                return x
        elif isinstance(orig_col,NoneType): # else, if column must not be selected, continue
            continue
        else:    # else, if a preprocessing function has been specified, together with column name, use it when extracting column
            fun = orig_col[1]
            orig_col = orig_col[0]
            
        # adjust reading parameters to ncbi format, if needed 
        try:
            ds_col = pd.read_csv(db_paths[db_name], on_bad_lines='skip', keep_default_na=False)[orig_col]
        except:
            ds_col = pd.read_csv(db_paths[db_name], names=ncbi_header, index_col=0, on_bad_lines='skip', sep=ncbi_sep, keep_default_na=False).reset_index(level=0)[orig_col]
        
        ## add column to the corresponding preprocessed db
        db[db_name][targ_col] = list(map(fun, ds_col)) # new_db col = preprocessed old_db col
    
    ## save each db
    pd.DataFrame.from_dict(db[db_name]).to_csv(os.path.join(output_path, db_name+'_table.csv'), index=False)


In [99]:
## Load previously saved datasets 
prep_dbs = dict()

for db_name in db_names_to_relevant_col_names.keys():
    prep_dbs[db_name] = pd.read_csv(os.path.join(output_path, db_name + '_table.csv'), on_bad_lines='skip', keep_default_na=False)

### (2) Link prep dataset terms to NCBI entities

In [100]:
db_names = list(str(x) for x in prep_dbs.keys()) # names of EPPO databases

### (3) Join Preprocessed Datasets

In [101]:
## prep ncbi db entities will be used to complete other database rows 
prep_ncbi_db = prep_dbs['ncbi']

In [102]:
## [EPPO-ONLY STEP] open both eppo (sci and com) dbs and concatenate them
prep_eppo_dbs = [prep_dbs[db_name] for db_name in db_names if 'eppo' in db_name]
prep_eppo_db = pd.concat(prep_eppo_dbs, ignore_index=True).fillna('')
display(prep_eppo_db.head())
len(prep_eppo_db)

## concatenate all dbs with matching name (normally only one in the case of EFSA)
prep_efsa_dbs = [prep_dbs[db_name] for db_name in db_names if 'efsa' in db_name]
prep_efsa_db = pd.concat(prep_efsa_dbs, ignore_index=True).fillna('')
display(prep_efsa_db.head())
len(prep_efsa_db)

,EPPO_CODE,preferred_name,synonym,ds_language
0,Acode,Asci,A,
1,Bcode,Bsci,B,bn
2,Bcode,Bsci,Bb,bn
3,Ccode,Csci,C,kg
4,Zcode,Zsci,Z,


,EFSA_PHT,preferred_name,re,match_language
0,Apht,,A,Albanian
1,Apht,,X,
2,Bpht,,B,
3,Dpht,,D,


4

In [103]:
ncbi_enriched_dbs = dict(
    efsa = fill_values_cross_db(prep_efsa_db,
                                    prep_ncbi_db,
                                    matching_columns=('re','synonym'),
                                    lookup_col='NCBI_TAXID'
                                    ),

    eppo = fill_values_cross_db(prep_eppo_db,
                                    prep_ncbi_db,
                                    matching_columns=('synonym','synonym'),
                                    lookup_col='NCBI_TAXID'
                                    )
)

In [104]:
for db_name,db in ncbi_enriched_dbs.items():
    db.to_csv(os.path.join(output_path, db_name+'_table.csv'), index=False)
    display(db)

,EFSA_PHT,preferred_name,re,match_language,NCBI_TAXID
0,Apht,,A,Albanian,Ataxid
1,Apht,,X,,
2,Bpht,,B,,
3,Dpht,,D,,Dtaxid


,EPPO_CODE,preferred_name,synonym,ds_language,NCBI_TAXID
0,Acode,Asci,A,,Ataxid
1,Bcode,Bsci,B,bn,
2,Bcode,Bsci,Bb,bn,
3,Ccode,Csci,C,kg,
4,Zcode,Zsci,Z,,


In [105]:
## check they loaded correctly (e.g. ncbi)
prep_dbs['ncbi'].head()

,NCBI_TAXID,preferred_name,synonym
0,Ataxid,Asci,A
1,Dtaxid,Dsci,D


#### Create empty DF with col names

In [106]:
empty_db = pd.DataFrame(columns=output_gloss_column_names)
display(empty_db)

,EPPO_CODE,EFSA_PHT,NCBI_TAXID,preferred_name,synonym,re,ds_language,match_language


df1 = pd.DataFrame([[3,4],[5,6]], columns=['a','b'])
df2 = pd.DataFrame([[5,6],[5,6]], columns=['a','c'])
display(df1)
pd.concat([df1,df2], ignore_index=True)

#### Add EPPO info to final DB

In [107]:
prep_eppo_db = ncbi_enriched_dbs['eppo']

In [108]:
# copy into a new db to which eppo information will be added 
eppo_final_db = empty_db.copy()

In [109]:
## loop over syns in EPPO db
for prep_eppo_db_i in tqdm(range(len(prep_eppo_db))):
    syn = str(prep_eppo_db['synonym'][prep_eppo_db_i]) 
    lang = prep_eppo_db['ds_language'][prep_eppo_db_i]
    key = prep_eppo_db['EPPO_CODE'][prep_eppo_db_i]
    sci_name = prep_eppo_db['preferred_name'][prep_eppo_db_i]
    taxid = prep_eppo_db['NCBI_TAXID'][prep_eppo_db_i]
    
    ## add collected info from NCBI and add it to row
    new_eppo_row = pd.DataFrame([[taxid, sci_name, syn, lang, key]], columns=['NCBI_TAXID', 'preferred_name', 'synonym', 'ds_language', 'EPPO_CODE'])
    
    ## add row to 
    eppo_final_db = pd.concat([eppo_final_db, new_eppo_row], ignore_index=True).fillna('')

100%|██████████| 5/5 [00:00<00:00, 307.76it/s]


In [110]:
eppo_final_db

,EPPO_CODE,EFSA_PHT,NCBI_TAXID,preferred_name,synonym,re,ds_language,match_language
0,Acode,,Ataxid,Asci,A,,,
1,Bcode,,,Bsci,B,,bn,
2,Bcode,,,Bsci,Bb,,bn,
3,Ccode,,,Csci,C,,kg,
4,Zcode,,,Zsci,Z,,,


In [111]:
eppo_final_db.to_csv('output.csv')

for i in range(10):
    print('i is:'+str(i))
    for j in range(10):
        print(j)
        if j%5 == 0 and j>0:
            print("break")
            break

#### Add EFSA info to final DB

EFSA is made of reg expression, while EPPO is made of synonyms.
In orfer to add EFSA information, to the database that is already populated with EPPO data, the regexes will be applied to the synonyms. If a match is found, the EFSA data will complete the previously EPPO-only row, else a new row will be created.

In [112]:
# copy into a new db to which eppo information will be added 
eppo_efsa_final_db = eppo_final_db.copy()
display(eppo_efsa_final_db)

prep_efsa_db = ncbi_enriched_dbs['efsa']
display(prep_efsa_db)

,EPPO_CODE,EFSA_PHT,NCBI_TAXID,preferred_name,synonym,re,ds_language,match_language
0,Acode,,Ataxid,Asci,A,,,
1,Bcode,,,Bsci,B,,bn,
2,Bcode,,,Bsci,Bb,,bn,
3,Ccode,,,Csci,C,,kg,
4,Zcode,,,Zsci,Z,,,


,EFSA_PHT,preferred_name,re,match_language,NCBI_TAXID
0,Apht,,A,Albanian,Ataxid
1,Apht,,X,,
2,Bpht,,B,,
3,Dpht,,D,,Dtaxid


In [113]:
prep_efsa_db.keys()

Index(['EFSA_PHT', 'preferred_name', 're', 'match_language', 'NCBI_TAXID'], dtype='object')

In [114]:
## loop over regexes in EFSA db
for prep_efsa_db_i in tqdm(range(len(prep_efsa_db))):
    match_found = False
    
    pattern = prep_efsa_db['re'][prep_efsa_db_i]
    taxid = prep_efsa_db['NCBI_TAXID'][prep_efsa_db_i]
    efsa_pht = prep_efsa_db['EFSA_PHT'][prep_efsa_db_i]
    match_lang =  prep_efsa_db['match_language'][prep_efsa_db_i]
    #synonym = prep_efsa_db['synonym'][prep_efsa_db_i]
    
    ## iterate over final db rows
    for eppo_efsa_final_db_i in range(len(eppo_efsa_final_db)):
        # info to be matched
        eppo_synonym = eppo_efsa_final_db['synonym'][eppo_efsa_final_db_i]
        eppo_taxid = eppo_efsa_final_db['NCBI_TAXID'][eppo_efsa_final_db_i]
        eppo_sci_name = eppo_efsa_final_db['preferred_name'][eppo_efsa_final_db_i]
        
        ## matching conditions
        #match_1 = eppo_synonym and eppo_synonym == synonym
        match_2= re.fullmatch(str(pattern), str(eppo_synonym))
        match_3 = eppo_taxid and taxid == eppo_taxid
    
        ## if re matches, add: re, efsa key, language, taxid
        sci_name = prep_efsa_db['preferred_name'][prep_efsa_db_i]
        
        ## check it's the same entity first      
        #if eppo_sci_name == sci_name:
        if match_2 or match_3:
            
            # assign info to matching row
            eppo_efsa_final_db['re'][eppo_efsa_final_db_i] = pattern
            eppo_efsa_final_db['EFSA_PHT'][eppo_efsa_final_db_i] = prep_efsa_db['EFSA_PHT'][prep_efsa_db_i]
            eppo_efsa_final_db['match_language'][eppo_efsa_final_db_i] = match_lang
            
            eppo_efsa_final_db['NCBI_TAXID'][eppo_efsa_final_db_i] = max(taxid, eppo_taxid)
            eppo_efsa_final_db['synonym'][eppo_efsa_final_db_i] = eppo_synonym
            match_found = True
        
    ## else, if no matches, add new line = re, syn, key, language, taxid
    if not match_found:
        pattern = prep_efsa_db['re'][prep_efsa_db_i]
        
        new_efsa_row = pd.DataFrame([[pattern, match_lang, efsa_pht, taxid]], columns=['re', 'match_language', 'EFSA_PHT', 'NCBI_TAXID'])
        display(new_efsa_row)
        eppo_efsa_final_db = pd.concat([eppo_efsa_final_db, new_efsa_row], ignore_index=True).fillna('')

eppo_efsa_final_db.head(10)

  0%|          | 0/4 [00:00<?, ?it/s]

,re,match_language,EFSA_PHT,NCBI_TAXID
0,X,,Apht,


,re,match_language,EFSA_PHT,NCBI_TAXID
0,D,,Dpht,Dtaxid


100%|██████████| 4/4 [00:00<00:00, 114.45it/s]


,EPPO_CODE,EFSA_PHT,NCBI_TAXID,preferred_name,synonym,re,ds_language,match_language
0,Acode,Apht,Ataxid,Asci,A,A,,Albanian
1,Bcode,Bpht,,Bsci,B,B,bn,
2,Bcode,,,Bsci,Bb,,bn,
3,Ccode,,,Csci,C,,kg,
4,Zcode,,,Zsci,Z,,,
5,,Apht,,,,X,,
6,,Dpht,Dtaxid,,,D,,


#### Copy complementary info to all terms representing the same entity

In [117]:
crossed_final_db = eppo_efsa_final_db.copy()

In [118]:
entity_spec_cols = ['EPPO_CODE', 'EFSA_PHT', 'NCBI_TAXID', 'preferred_name']

for col_i in range(len(entity_spec_cols)):
    matching_col = entity_spec_cols[col_i]
    filling_cols = entity_spec_cols.copy()
    filling_cols.remove(matching_col)
    
    for i in range(len(crossed_final_db)): # row to be filled
        for j in range(len(crossed_final_db)): # row to take complementary info
            if i == j:
                break
            
            ## if matching values on matching_col (and not empty)
            if crossed_final_db.at[i,matching_col] and crossed_final_db.at[i,matching_col] == crossed_final_db.at[j,matching_col]:
                
                print('match found in column: '+matching_col)    
                print(crossed_final_db.at[i,matching_col])
                print(crossed_final_db.at[j,matching_col])
                print('rows:',i,j)
                
                ## fill with values from complementary row
                for filling_col in filling_cols:
                    crossed_final_db.at[i,filling_col] = max(
                        crossed_final_db.at[i,filling_col],
                        crossed_final_db.at[j,filling_col]
                    )
                if i == 4:
                    print(j)
                    print(crossed_final_db.at[j,filling_col])
    
display(crossed_final_db)

match found in column: EPPO_CODE
Bcode
Bcode
rows: 2 1
match found in column: EFSA_PHT
Bpht
Bpht
rows: 2 1
match found in column: EFSA_PHT
Apht
Apht
rows: 5 0
match found in column: NCBI_TAXID
Ataxid
Ataxid
rows: 5 0
match found in column: preferred_name
Bsci
Bsci
rows: 2 1
match found in column: preferred_name
Asci
Asci
rows: 5 0


,EPPO_CODE,EFSA_PHT,NCBI_TAXID,preferred_name,synonym,re,ds_language,match_language
0,Acode,Apht,Ataxid,Asci,A,A,,Albanian
1,Bcode,Bpht,,Bsci,B,B,bn,
2,Bcode,Bpht,,Bsci,Bb,,bn,
3,Ccode,,,Csci,C,,kg,
4,Zcode,,,Zsci,Z,,,
5,Acode,Apht,Ataxid,Asci,,X,,
6,,Dpht,Dtaxid,,,D,,


In [ ]:
crossed_final_db.to_csv(LR_filepath)